# Pneumonia Final Project Playground

> *This is a testing environment for assessing the different models*

In [ ]:
!pip install gdown

In [ ]:
import os

from keras.saving import load_model
from pathlib import Path
import tensorflow as tf
import numpy as np
import pickle
import zipfile

from google.colab import files as colabfiles

MODELS_DIR = r'/content/old_models'
SIZE = 150

def download_file(file_id, destination):
  !gdown --id $file_id -O $destination


def download_model(file_id, destination):
  temp_zip = f'{destination}.zip'
  download_file(file_id, temp_zip)

  with zipfile.ZipFile(temp_zip, 'r') as zip_ref:
    zip_ref.extractall(destination)

  os.remove(temp_zip)


def process_image_for_prediction(size = SIZE):
  image_string = list(colabfiles.upload().values())[0]
  image = tf.io.decode_image(image_string, channels=1)
  image = tf.image.resize(image, (size, size))
  image_tensor = tf.expand_dims(image, axis=0)
  binary_array = np.asarray(image_tensor)
  binary_array.reshape(1,-1)
  binary_array = binary_array / 255

  return binary_array


MODEL_1 = 'MODEL_1'
MODEL_2 = 'MODEL_2'
EMBEDDING_MODEL_1 = 'EMBEDDING_MODEL_1'
EMBEDDING_MODEL_2 = 'EMBEDDING_MODEL_2'
KNN_1 = 'KNN_1'
KNN_2 = 'KNN_2'
MODEL_4 = 'MODEL_4'

In [ ]:
download_model('1KuDRHUAqib308xn-ST76dh8XquvmAQ6R', MODEL_1)
download_model('1-S2tt6xIZGbQfPsygvneeUOnI7HozKik', MODEL_2)
download_model('1SQ-L3vze24J7DsqYHuWGMLC_TAu4Ls9G', EMBEDDING_MODEL_1)
download_model('18TEWxyvnbQGXXZuRSuWM8JXjp84z2JFf', EMBEDDING_MODEL_2)
download_file('1yys_C3ljg1WmCfTdiyWmiZ71EfJ4TjT-', KNN_1)
download_file('1E6az9MymUQk45cYdBBEU0KrSGjmWJFCi', KNN_2)
download_model('1_sNV928uSerQtDv74hx4zz9Q_j_p5G9N', MODEL_4)

# Task 1 - Playground
In this assignment, we were tasked with constructing a neural network designed to differentiate between images depicting lungs afflicted with pneumonia and those that are not.

In [ ]:
TASK_1_CLASSES = ['NORMAL', 'PNEUMONIA']
model_1 = load_model(MODEL_1)

image = process_image_for_prediction()
prediction = int(np.round(model_1.predict(image)).flatten()[0])
print(TASK_1_CLASSES[prediction])

# Task 2 - Playground
This task resembles the previous one, but it requires distinguishing between two types of pneumonia: viral and bacterial.

In [ ]:
TASK_2_CLASSES = ['BACTERIA', 'NORMAL', 'VIRUS']
model_2 = load_model(MODEL_2)

image = process_image_for_prediction()
prediction = np.argmax(model_2.predict(image).flatten())
print(TASK_2_CLASSES[prediction])

# Task 3 - Playground
Here, we aim to employ the KNN strategy to facilitate the classification of new images by leveraging both the first and second neural networks developed in the previous tasks.

In [ ]:
TASK_3_1_CLASSES = ['NORMAL', 'PNEUMONIA']
embedding_model_1 = load_model(EMBEDDING_MODEL_1)

with open(KNN_1, 'rb') as f:
    knn1 = pickle.load(f)
    image = process_image_for_prediction()
    embedding_prediction = embedding_model_1.predict(image)
    prediction = knn1.predict(embedding_prediction)[0]

print(TASK_3_1_CLASSES[prediction])

In [ ]:
TASK_3_2_CLASSES = ['BACTERIA', 'NORMAL', 'VIRUS']
embedding_model_2 = load_model(EMBEDDING_MODEL_2)

with open(KNN_2, 'rb') as f:
    knn2 = pickle.load(f)
    image = process_image_for_prediction()
    embedding_prediction = embedding_model_2.predict(image)
    prediction = knn2.predict(embedding_prediction)[0]

print(TASK_3_2_CLASSES[prediction])

# Task 4 - Playground
In this task, we operate under the assumption that we only possess "normal" lung scans. Our objective is to identify pneumonia scans utilizing the anomaly detection method.

In [ ]:
THRESHOLD = 0.02
def detect_anomaly(img_array, model, threshold):
    reconstructed_img = model.predict(img_array)
    loss = np.mean(np.abs(reconstructed_img - img_array))
    print(loss)
    if loss > threshold:
        print("PNEUMONIA")
    else:
        print("NORMAL")

img = process_image_for_prediction(size=256)
autoencoder = load_model(MODEL_4)
detect_anomaly(img, autoencoder, THRESHOLD)